In [2]:
import pandas as pd
import numpy as np

In [3]:
%%time
df = pd.read_csv("exp.csv",names=['edate','statement','dr','cr','bal'],header=0)
df["edate"] = pd.to_datetime(df["edate"],format="%d-%m-%Y")
df[["dr", "cr"]] = df[["dr", "cr"]].replace(" ",0).apply(pd.to_numeric)

df =  df[df['edate'].dt.year==2018]
df.tail(10)

CPU times: user 36.1 ms, sys: 7.96 ms, total: 44 ms
Wall time: 182 ms


,edate,statement,dr,cr,bal
452,2018-12-30,ATM-CASH/BELAGO SHRIRANGAPA/Belagola/301218,4000.00,0.0,5583.80
453,2018-12-30,CASH-REVERSAL-ATM/836413000590/301218/13:58,0.00,4000.0,9583.80
454,2018-12-30,ECOM PUR/LANDMARK ONLI/MUMBAI/301218/08:50,449.00,0.0,9134.80
455,2018-12-31,917010081781236:Int.Pd:01-10-2018 to 31-12-2018,0.00,101.0,9235.80
456,2018-12-31,NEFT/CMS992301009/INFOSYS LIMITED/INFY SALARY FOR,0.00,22103.0,31338.80
457,2018-12-31,UPI/P2A/836510901163/MANU B E,0.00,3000.0,34338.80
458,2018-12-31,UPI/P2A/836510995442/RAJIV P,0.00,250.0,34588.80
459,2018-12-31,POS/SHUBHAM RESTAURAN/MYSORE/311218/06:57,28.00,0.0,34560.80
460,2018-12-31,ATM-CASH-AXIS/T3RH278801/3129/311218/MYSORE,10000.00,0.0,24560.80
461,2018-12-31,POS/LOYAL WORLD/MYSORE/311218/12:48,53.35,0.0,24507.45


In [4]:
df.dtypes

edate        datetime64[ns]
statement            object
dr                  float64
cr                  float64
bal                 float64
dtype: object

In [5]:
exp_df = df[['edate','dr','cr']]

exp_df = exp_df.groupby(exp_df['edate'].dt.month,as_index=False).agg(['sum'])
exp_df.columns = exp_df.columns.map('_'.join)
exp_df.insert(0,'month',exp_df.index)
exp_df.to_json(orient='records')

'[{"month":1,"dr_sum":0.0,"cr_sum":16070.0},{"month":2,"dr_sum":22770.0,"cr_sum":15700.0},{"month":3,"dr_sum":24195.13,"cr_sum":27849.0},{"month":4,"dr_sum":29285.94,"cr_sum":21634.0},{"month":5,"dr_sum":6441.65,"cr_sum":15651.0},{"month":6,"dr_sum":21951.95,"cr_sum":27146.41},{"month":7,"dr_sum":18471.45,"cr_sum":23329.0},{"month":8,"dr_sum":28961.6,"cr_sum":22454.0},{"month":9,"dr_sum":55511.54,"cr_sum":38483.0},{"month":10,"dr_sum":16390.66,"cr_sum":39392.5},{"month":11,"dr_sum":40990.34,"cr_sum":38394.25},{"month":12,"dr_sum":58643.45,"cr_sum":62018.0}]'

In [18]:
import re

def filter(str):
    strList = str.split("/")
    copy_List = []
    for l in strList:
        if not bool(re.search(r'\d', l)):
            copy_List.append(l.replace(" ","_"))
    return " ".join(copy_List)
        
#     return " ".join([l.replace(" ","_") for l in strList])
    

In [19]:
df_month =  df[df['edate'].dt.month==4]
df_month.loc[:,'statement'] = df_month['statement'].apply(filter)
df_month.tail(10)
# pd.Series(' '.join(df_month['statement']).lower().split("/")).value_counts()[:100]

,edate,statement,dr,cr,bal
92,2018-04-23,ECOM_PUR PAYTM Noida,200.0,0.0,283.93
93,2018-04-24,UPI Mr_VYSHAK__G,0.0,1000.0,1283.93
94,2018-04-24,UPI RAJIV__P,1000.0,0.0,283.93
95,2018-04-25,ECOM_PUR PAYTM Noida,100.0,0.0,183.93
96,2018-04-26,UPI RAJIV__P,0.0,1000.0,1183.93
97,2018-04-26,UPI Mr_VYSHAK__G,1000.0,0.0,183.93
98,2018-04-27,NEFT INFOSYS_LIMITED INFY_SALARY_FOR,0.0,13959.0,14142.93
99,2018-04-27,UPI Mr_VYSHAK__G,7641.0,0.0,6501.93
100,2018-04-30,ATM-CASH BELAGULA BELAGULA,500.0,0.0,6001.93
101,2018-04-30,ECOM_PUR PAYTM NOIDA,1000.0,0.0,5001.93


In [20]:
pd.Series(' '.join(df_month['statement']).upper().split(" ")).value_counts()[:100].head(25)

UPI                  11
ECOM_PUR             10
PAYTM                 7
NOIDA                 7
MYSORE                6
MR_VYSHAK__G          4
RAJIV__P              4
POS                   4
ATM-CASH              3
GURGAON               2
BELAGULA              2
HEBBAL_INDUSTRIAL     2
BOOKMYSHOW_CO         2
S.PIZZA_HUT_SAP       1
ADITYA_BIRLA_RETA     1
NEFT                  1
LIFE_STYLE_INTL_P     1
M                     1
VISHAL                1
INFY_SALARY_FOR       1
MANU_B_E              1
KFC                   1
PHONEPE               1
MYNTRA_DESIGN         1
INFOSYS_LIMITED       1
dtype: int64